In  a docker terminal run
jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser --allow-root

then open jupyter in the printed link (such as http://127.0.0.1:8888/tree?token=4d9d578397aede50fc5bd2d92794b562a6bbcbc73b2dfe51)

CODE HERE, REFRESH AND EXECUTE IN THE BROWSER


from swn import models as sm
from buek import models as bm
from 

In [8]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
sys.path.append('/app')

os.environ['DJANGO_SETTINGS_MODULE'] = 'app.settings'
os.environ['PYTHONPATH'] = '/app'

django.setup()

In [43]:
from swn import models as sm
from buek import models as buek_models
from monica import models as mm
import json
from datetime import datetime
from django.forms.models import model_to_dict

In [22]:
def soil_profiles_from_polygon_ids(soil_profile_polygon_ids):
    """
    Get soil profiles from polygon ids.
    """
    start = datetime.now()
    unique_land_usages = buek_models.SoilProfile.objects.filter(
        polygon_id__in=soil_profile_polygon_ids
    ).values_list('landusage_corine_code', 'landusage').distinct()
    # Filter out water (code 51)
    land_usage_choices = {code: usage for code, usage in unique_land_usages if code != 51}
    
    # Initialize data_json with land usage codes
    data_json = {code: {} for code in land_usage_choices.keys()}

    soil_data = buek_models.SoilProfileHorizon.objects.select_related('soilprofile').filter(
        soilprofile__polygon_id__in=soil_profile_polygon_ids
        ).order_by('soilprofile__landusage_corine_code', 'soilprofile__system_unit', 'soilprofile__area_percentage', 'horizont_nr')

    # Loop through soil data and populate data_json
    
    for item in soil_data:
        try:
            if item.soilprofile.landusage_corine_code != 51:
                land_code = item.soilprofile.landusage_corine_code
                system_unit = item.soilprofile.system_unit
                area_percentage = item.soilprofile.area_percentage
                profile_id = item.soilprofile.id
                horizon_nr = item.horizont_nr

                if system_unit not in data_json[land_code]:
                    data_json[land_code][system_unit] = {
                        'area_percentages': set(),
                        'soil_profiles': {}
                    }

                data_json[land_code][system_unit]['area_percentages'].add(area_percentage)

                if area_percentage not in data_json[land_code][system_unit]['soil_profiles']:
                    data_json[land_code][system_unit]['soil_profiles'][area_percentage] = {}

                if profile_id not in data_json[land_code][system_unit]['soil_profiles'][area_percentage]:
                    data_json[land_code][system_unit]['soil_profiles'][area_percentage][profile_id] = {'horizons': {}}

                data_json[land_code][system_unit]['soil_profiles'][area_percentage][profile_id]['horizons'][horizon_nr] = {
                    'obergrenze_m': item.obergrenze_m,
                    'untergrenze_m': item.untergrenze_m,
                    'stratigraphie': item.stratigraphie,
                    'herkunft': item.herkunft,
                    'geogenese': item.geogenese,
                    # 'fraktion': item.fraktion,
                    'summe': item.summe,
                    # 'gefuege': item.gefuege,
                    # 'torfarten': item.torfarten,
                    # 'substanzvolumen': item.substanzvolumen,
                    'bulk_density_class': item.bulk_density_class.bulk_density_class if item.bulk_density_class_id is not None else 'no data',
                    'bulk_density': item.bulk_density_class.raw_density_g_per_cm3 if item.bulk_density_class_id is not None else 'no data',
                    'humus_class': item.humus_class.humus_class if item.humus_class_id is not None else 'no data',
                    'humus_corg': item.humus_class.corg if item.humus_class_id is not None else 'no data',
                    'ka5_texture_class': item.ka5_texture_class.ka5_soiltype if item.ka5_texture_class_id is not None else 'no data',
                    'sand': round(item.ka5_texture_class.sand, 2) if item.ka5_texture_class_id is not None else 'no data',
                    'clay': round(item.ka5_texture_class.clay, 2) if item.ka5_texture_class_id is not None else 'no data',
                    'silt': round(item.ka5_texture_class.silt, 2) if item.ka5_texture_class_id is not None else 'no data',
                    'ph_class': item.ph_class.ph_class if item.ph_class_id is not None else 'no data',
                    'ph_lower_value': item.ph_class.ph_lower_value if item.ph_class_id is not None else 'no data',
                    'ph_upper_value': item.ph_class.ph_upper_value if item.ph_class_id is not None else 'no data',                    
                }
        except Exception as e:
            print("Exception occurred:", e)
           
    # Sort area percentages
    for land_code in data_json:
        for system_unit in data_json[land_code]:
            data_json[land_code][system_unit]['area_percentages'] = sorted(list(data_json[land_code][system_unit]['area_percentages']))

    data_menu = {
        # 'soil_profile_form': forms.SoilProfileSelectionForm().set_choices(land_usage_choices),
        'text': 'name',
        'id': 1,
        'polygon_ids': soil_profile_polygon_ids,
        'system_unit_json': json.dumps(data_json),
        'landusage_choices': json.dumps(land_usage_choices),
    }
    print('Time elapsed: ', datetime.now() - start)
    return data_menu

In [23]:
sp_ids = [475027, 475028]

In [24]:
soil_profiles_from_polygon_ids(sp_ids)

Time elapsed:  0:00:00.338861


{'text': 'name',
 'id': 1,
 'polygon_ids': [475027, 475028],
 'system_unit_json': '{"21": {"BB-GG": {"area_percentages": [5], "soil_profiles": {"5": {"18833": {"horizons": {"1": {"obergrenze_m": 0.0, "untergrenze_m": 0.3, "stratigraphie": "qp", "herkunft": "Sf", "geogenese": "f", "summe": 1.0, "bulk_density_class": "Ld3", "bulk_density": 1.7, "humus_class": "h2", "humus_corg": 0.8720930218696594, "ka5_texture_class": "Ss", "sand": 0.93, "clay": 0.02, "silt": 0.05, "ph_class": "s1", "ph_lower_value": 6.1, "ph_upper_value": 6.8}, "2": {"obergrenze_m": 0.3, "untergrenze_m": 0.35, "stratigraphie": "qp", "herkunft": "Sf", "geogenese": "f", "summe": 1.0, "bulk_density_class": "Ld3", "bulk_density": 1.7, "humus_class": "h1", "humus_corg": 0.29069799184799194, "ka5_texture_class": "Ss", "sand": 0.93, "clay": 0.02, "silt": 0.05, "ph_class": "s1", "ph_lower_value": 6.1, "ph_upper_value": 6.8}, "3": {"obergrenze_m": 0.35, "untergrenze_m": 0.5, "stratigraphie": "qp", "herkunft": "Sf", "geogenese":

In [59]:
def soil_profiles_from_polygon_ids_new(soil_profile_polygon_ids):
    """
    Get soil profiles from polygon ids.
    """
    start = datetime.now()

    soil_profiles = buek_models.SoilProfile.objects.filter(polygon_id__in=soil_profile_polygon_ids).exclude(landusage_corine_code=51)
    unique_land_usages = soil_profiles.values_list('landusage_corine_code', 'landusage').distinct()
    # Filter out water (code 51)
    land_usage_choices = {code: usage for code, usage in unique_land_usages}
    
    # Initialize data_json with land usage codes
    data_json = {code: {} for code in land_usage_choices.keys()}

    soil_data = buek_models.SoilProfileHorizon.objects.select_related('soilprofile').filter(
        soilprofile__in=soil_profiles
        ).order_by('soilprofile__landusage_corine_code', 'soilprofile__system_unit', 'soilprofile__area_percentage', 'horizont_nr')

    # Loop through soil data and populate data_json
    
    for item in soil_data:
        try:
            
            land_code = item.soilprofile.landusage_corine_code
            system_unit = item.soilprofile.system_unit
            area_percentage = item.soilprofile.area_percentage
            profile_id = item.soilprofile.id
            horizon_nr = item.horizont_nr

            if system_unit not in data_json[land_code]:
                data_json[land_code][system_unit] = {
                    'area_percentages': set(),
                    'soil_profiles': {}
                }

            data_json[land_code][system_unit]['area_percentages'].add(area_percentage)

            if area_percentage not in data_json[land_code][system_unit]['soil_profiles']:
                data_json[land_code][system_unit]['soil_profiles'][area_percentage] = {}

            if profile_id not in data_json[land_code][system_unit]['soil_profiles'][area_percentage]:
                data_json[land_code][system_unit]['soil_profiles'][area_percentage][profile_id] = {'horizons': {}}

            data_json[land_code][system_unit]['soil_profiles'][area_percentage][profile_id]['horizons'][horizon_nr] = {
                'obergrenze_m': item.obergrenze_m,
                'untergrenze_m': item.untergrenze_m,
                'stratigraphie': item.stratigraphie,
                'herkunft': item.herkunft,
                'geogenese': item.geogenese,
                'summe': item.summe,
                'bulk_density_class': item.bulk_density_class.bulk_density_class if item.bulk_density_class_id is not None else 'no data',
                'bulk_density': item.bulk_density_class.raw_density_g_per_cm3 if item.bulk_density_class_id is not None else 'no data',
                'humus_class': item.humus_class.humus_class if item.humus_class_id is not None else 'no data',
                'humus_corg': item.humus_class.corg if item.humus_class_id is not None else 'no data',
                'ka5_texture_class': item.ka5_texture_class.ka5_soiltype if item.ka5_texture_class_id is not None else 'no data',
                'sand': round(item.ka5_texture_class.sand, 2) if item.ka5_texture_class_id is not None else 'no data',
                'clay': round(item.ka5_texture_class.clay, 2) if item.ka5_texture_class_id is not None else 'no data',
                'silt': round(item.ka5_texture_class.silt, 2) if item.ka5_texture_class_id is not None else 'no data',
                'ph_class': item.ph_class.ph_class if item.ph_class_id is not None else 'no data',
                'ph_lower_value': item.ph_class.ph_lower_value if item.ph_class_id is not None else 'no data',
                'ph_upper_value': item.ph_class.ph_upper_value if item.ph_class_id is not None else 'no data',                    
            }
        except Exception as e:
            print("Exception occurred:", e)
           
    # Sort area percentages
    for land_code in data_json:
        for system_unit in data_json[land_code]:
            data_json[land_code][system_unit]['area_percentages'] = sorted(list(data_json[land_code][system_unit]['area_percentages']))

    data_menu = {
        'text': 'name',
        'id': 1,
        'polygon_ids': soil_profile_polygon_ids,
        'system_unit_json': json.dumps(data_json),
        'landusage_choices': json.dumps(land_usage_choices),
    }
    print('Time elapsed: ', datetime.now() - start)
    return data_menu

In [60]:
data = soil_profiles_from_polygon_ids_new(sp_ids)
data

Time elapsed:  0:00:00.260970


{'text': 'name',
 'id': 1,
 'polygon_ids': [475027, 475028],
 'system_unit_json': '{"21": {"BB-GG": {"area_percentages": [5], "soil_profiles": {"5": {"18833": {"horizons": {"1": {"obergrenze_m": 0.0, "untergrenze_m": 0.3, "stratigraphie": "qp", "herkunft": "Sf", "geogenese": "f", "summe": 1.0, "bulk_density_class": "Ld3", "bulk_density": 1.7, "humus_class": "h2", "humus_corg": 0.8720930218696594, "ka5_texture_class": "Ss", "sand": 0.93, "clay": 0.02, "silt": 0.05, "ph_class": "s1", "ph_lower_value": 6.1, "ph_upper_value": 6.8}, "2": {"obergrenze_m": 0.3, "untergrenze_m": 0.35, "stratigraphie": "qp", "herkunft": "Sf", "geogenese": "f", "summe": 1.0, "bulk_density_class": "Ld3", "bulk_density": 1.7, "humus_class": "h1", "humus_corg": 0.29069799184799194, "ka5_texture_class": "Ss", "sand": 0.93, "clay": 0.02, "silt": 0.05, "ph_class": "s1", "ph_lower_value": 6.1, "ph_upper_value": 6.8}, "3": {"obergrenze_m": 0.35, "untergrenze_m": 0.5, "stratigraphie": "qp", "herkunft": "Sf", "geogenese":

In [36]:
soil_profiles = buek_models.SoilProfile.objects.filter(polygon_id__in=sp_ids).exclude(landusage_corine_code=51)

In [47]:
unique_land_usages = soil_profiles.values_list('landusage_corine_code', 'landusage').distinct()

In [53]:
land_usage_choices = {code: usage for code, usage in unique_land_usages}
land_usage_choices

{21: 'Ackerflächen', 23: 'Grünland', 31: 'Wälder'}

In [62]:
data_json = {code: {} for code in land_usage_choices.keys()}

In [63]:
data_json.keys()

dict_keys([21, 23, 31])

In [56]:
soil_data = buek_models.SoilProfileHorizon.objects.select_related('soilprofile').filter(
        soilprofile__in=soil_profiles
        ).order_by('soilprofile__landusage_corine_code', 'soilprofile__system_unit', 'soilprofile__area_percentage', 'horizont_nr')

In [67]:
json.loads(data['system_unit_json']).keys()

dict_keys(['21', '23', '31'])